In [ ]:
from generator import Task, LinearModel
from tokenizer import FloatTokenizer
from dataset import LinearModelDataset
from config import getConfig
from train import Trainer

In [ ]:
cfg = getConfig()
trainer = Trainer(cfg)

In [ ]:
train_dl, val_dl = trainer.getDataset(cfg, 1)

In [ ]:
trainer.runEpoch(1, seed=1, train=False)

In [ ]:
# 3e6 // 50
